In [1]:
# (C) Copyright 1996- ECMWF.
#
# This software is licensed under the terms of the Apache Licence Version 2.0
# which can be obtained at http://www.apache.org/licenses/LICENSE-2.0.
# In applying this licence, ECMWF does not waive the privileges and immunities
# granted to it by virtue of its status as an intergovernmental organisation
# nor does it submit to any jurisdiction.

In [2]:
from pathlib import Path # for dictionaries/files

# basic libraries for data analysis
import numpy as np 
import pandas as pd 
import xarray as xr

# plotting functions
from matplotlib import rc

import matplotlib.pyplot as plt
import seaborn as sns
import cartopy
import cartopy.crs as ccrs

# functions/classes used for specific plots
from matplotlib.colors import ListedColormap, BoundaryNorm # for user-defined colorbars on matplotlib
from mpl_toolkits.axes_grid1 import AxesGrid # for multiplots and nice mixing with cartopy
from cartopy.mpl.geoaxes import GeoAxes # for adding cartopy attributes to subplots
from matplotlib.image import imread
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

In [3]:
rc('font', size=8)
rc('axes', labelsize=7, linewidth=0.2)
rc('legend', fontsize=6)
rc('xtick', labelsize=6)
rc('ytick', labelsize=6)
rc('lines', lw=0.5, mew=0.4)
rc('grid', linewidth=0.2)

sns.set_palette('colorblind') # change the default patette to colorblind-friendly

In [4]:
input_dir = '/ProcessedData/'
output_dir = '/PlotsPaper/'

Path(output_dir).mkdir(parents=True, exist_ok=True) # generate subfolder for storing the results
Path(output_dir+'Png/').mkdir(parents=True, exist_ok=True) # generate subfolder for storing png figures
Path(output_dir+'Pdf/').mkdir(parents=True, exist_ok=True) # generate subfolder for storing pdf figures

### Auxiliary data

In [5]:
# Auxiliary functions for informative colorbars

def plot_discrete_bar_inset(col_lims, col_used, data_used, ax, width, height, xlabel, cat_names, 
                            col_text='black', alpha_used=1, loc=8, brdpad=-2, bar_text_size=6):
    
    ' Categorical data: add bar plot as colorbar and include names of classes and percentage of data per class'
    
    if type(col_text) == str: col_text = [col_text]*len(col_used)
    
    ax_sub = inset_axes(ax, loc=loc, width=width, height=height, borderpad=brdpad) # create the axes of the bar

    Cl = col_lims[::-1] # reverse because barplot starts from largest to smallest
    Cu = col_used[::-1] # reverse because barplot starts from largest to smallest

    ax_sub.barh(width=Cl-Cl[-1], y=0, height=1, color=Cu, alpha=alpha_used) # plot colored bars
    ax_sub.barh(width=Cl[0]-Cl[-1], y=0, height=1,
                edgecolor='black', color='none', linewidth=.2, clip_on=False) # transparent bar with black frame

    ax_sub.set_xlim(col_lims[0], col_lims[-1]) # change the limits of the plot
    ax_sub.set_ylim(-.5, .5) # change so that colorbar covers full plot (colorbar has y=0, and height=1)
    ax_sub.set_yticks([]) # remove yticks
    
    Xticks = pd.Series(col_lims).rolling(2).mean().values[1:] # get position of ticks as middle point of each bar
    for i, j in enumerate(cat_names): # inside the bars in the Xticks locations state the provided cat_names
        ax_sub.text(s=j, x=Xticks[i], y=0, ha='center', va='center', color=col_text[i], size=bar_text_size)

    ax_sub.set_xticks(ticks=Xticks) # add the ticks in the desired locations
    ax_sub.set_xticklabels(labels=data_used, fontsize=6) # add the xticks_labels
    ax_sub.set_xlabel(xlabel, size=6) # add the final title of the bar as xlabel
    ax_sub.xaxis.set_tick_params(width=.2, length=1)
    
    return ax_sub

def plot_continuous_bar_inset(col_lims, col_used, data, ax, width, height, xlabel, loc=8, brdpad=-2):
    
    ' Continuous data: Add boxplot inside colorbar with the distribution of the studied variable for all the points'
    
    ax_sub = inset_axes(ax, loc=loc, width=width, height=height, borderpad=brdpad) # create the axes of the bar
    
    Cl = col_lims[::-1] # reverse because barplot starts from largest to smallest
    Cu = col_used[::-1] # reverse because barplot starts from largest to smallest
    
    ax_sub.barh(width=Cl-Cl[-1], y=0, height=1, left=Cl[-1], color=Cu) # plot colored bars
    ax_sub.barh(width=Cl[0]-Cl[-1], y=0, height=1, left=Cl[-1],
                edgecolor='black', color='none', linewidth=.2, clip_on=False) # transparent bar with back frame

    # plot boxplot of data; use positions=[0], so that the plot aligns with the colorbars
    ax_sub.boxplot(data, sym='x', vert=False, widths=.3, positions=[0], capprops={'linewidth': .2}, 
                    boxprops={'linewidth': .2}, flierprops={'marker': 'x', 'markersize': .3}, 
                    medianprops={'color': 'black', 'linewidth': .2}, whiskerprops={'linewidth': .2})

    ax_sub.set_ylim(-0.5, 0.5) # change so that colorbar covers full plot (colorbar has y=0, and height=1)
    ax_sub.set_yticks([]) # remove ticks of y-axis
    
    ax_sub.set_xlim(col_lims[0], col_lims[-1]) # set limits of x-axis
    ax_sub.set_xticks(col_lims) # set ticks of x axis
    ax_sub.xaxis.set_tick_params(width=.2, length=1) # change ticks visualization (same width as linewidth of barh)
    ax_sub.set_xlabel(xlabel, size=6) # set label of x-axis
    
    return ax_sub

In [6]:
# auxiliary variables for patterns (names, colors, etc)
New_names = ['Atlantic Low', 'Biscay Low', 'Iberian Low', 'Sicilian Low', 'Balkan Low', 'Black Sea Low',
             'Mediterranean High', 'Minor Low', 'Minor High'] # naming in the final order of interest

Cl_cols_used = [(.02, 0.40, 0.55), (.0, 0.75, 0.7), (.0, 0.63, 0.4), (.85, 0.35, 0.1), (.8, 0.5, 0.75), 
                (.7, 0.5, 0.45), (1, 0.7, 0.75), (.65, 0.65, 0.65), (.95, 0.95, 0.2)] # color for each pattern
Cl_vals_used = np.arange(len(Cl_cols_used))

# final area used for analysis
Area_used = [48, -10, 27, 41]
Y_max, X_min, Y_min, X_max = Area_used

# areas for subplots of Figures 8 and 10
Areas = {'All': [X_min, X_max, Y_min, Y_max], 
         'Iberia': [-9, -1, 37, 44], 'Northern Morocco': [-7, -4, 34, 36], 
         'Southern Italy': [15, 18.5, 38, 41], 'Alpes': [8, 13, 45, 46], 'Aegean - Western Turkey': [26, 30, 36, 41]}

### Figure 1

In [7]:
Composites = xr.open_dataset(input_dir+'PatternComposites_ERA5.nc')
Composites = Composites.to_array()
Labels = pd.read_csv(input_dir+'PatternAllocations_ERA5.csv', index_col=0)
Labels.index = pd.to_datetime(Labels.index)
Labels = Labels[Labels.index<=pd.to_datetime('2019-12-31')]['Label'] # only 1979-2019, as in Mastrantonas, et al 2021
Climat_Frequencies = Labels.value_counts()/len(Labels)*100
Climat_Frequencies = Climat_Frequencies.reindex(range(len(New_names)))

In [8]:
title_aux = list(map(chr, range(97, 123)))[:9] # alphabetic order for subplots naming
Colors_used = sns.color_palette('RdBu_r', n_colors=15)
Colors_used = Colors_used[:5]+['white']+Colors_used[-5:]
Colors_used = ListedColormap(Colors_used)
Colors_limits = [-16, -13, -10, -7, -4, -1, 1, 4, 7, 10, 13, 16] # levels for the colors (actual abs. max is 15.7)
Cont_levels = np.linspace(-24, 24, 13) # act. max is 21.2

X, Y = np.meshgrid(Composites.longitude, Composites.latitude)
Max_values = np.abs(Composites).max(dim=['cluster', 'latitude', 'longitude']).values
Max_values = np.round(Max_values/np.array([100, 98.1]), 2)
print('Absolute Max values for SLP and Z500 are {} hPa and {} dam respectively'.format(Max_values[0], Max_values[1]))

In [9]:
axes_class = (GeoAxes, dict(map_projection=ccrs.PlateCarree()))
fig = plt.figure(figsize=(18/2.54, 9.1/2.54))
grid = AxesGrid(fig, 111, nrows_ncols=(3,3), axes_pad=.2,
                cbar_mode='single', cbar_location='right', cbar_pad=.1,
                axes_class=axes_class, cbar_size='3%', label_mode='')

for i, ax in enumerate(grid):

    ax.set_extent([X.min(), X.max(), Y.min(), Y.max()], crs=ccrs.PlateCarree()) # set extent
    ax.outline_patch.set_linewidth(.2) # reduce the border thickness
    
    ax.coastlines(resolution='110m', linewidth=.5, color='grey') # add coastline

    contf = ax.contourf(X, Y, Composites.sel(variable='SLP', cluster=i)/100, # plot contourf for SLP anomalies
                        transform=ccrs.PlateCarree(), levels=Colors_limits, cmap=Colors_used) 
    cont = ax.contour(X, Y, Composites.sel(variable='Z500', cluster=i)/98.1, # plot contour for Z500 anomalies
                      transform=ccrs.PlateCarree(),levels=Cont_levels, colors='black') 
    ax.clabel(cont, inline=1, fontsize=6, fmt='%d')
    for line, lvl in zip(cont.collections, cont.levels):
        if lvl == 0:
            line.set_linestyle(':')

    ax.set_title('({}) Cl. {} - {} ({:.1f}%)'.format(title_aux[i], i+1, New_names[i], Climat_Frequencies[i]), 
                 pad=4, size=7.5, loc='left')

cbar = ax.cax.colorbar(contf, ticks=Colors_limits, spacing='proportional') # add colorbar
cbar.ax.set_title("SLP' (hPa)", size=6.5, loc='left')
cbar.ax.yaxis.set_tick_params(width=.25, length=3)

plt.subplots_adjust(left=0.02, bottom=0.01, right=.95, top=.96)
fig.savefig(output_dir+'Pdf/Fig1.pdf')
fig.savefig(output_dir+'Png/Fig1.png', dpi=600, transparent=True)

del(axes_class, fig, grid, i, ax, contf, cont, line, lvl, cbar)

In [10]:
del(Composites, Labels, Climat_Frequencies, title_aux, Colors_used, Colors_limits, Cont_levels, X, Y, Max_values)

### Figure 2

In [11]:
fig, ax_all = plt.subplots(1, 1, figsize=(8/2.54, 3.7/2.54), subplot_kw={'projection': ccrs.PlateCarree()})


ax_all.set_extent([X_min, X_max, Y_min, Y_max], crs=ccrs.PlateCarree())
ax_all.outline_patch.set_linewidth(.2) # reduce the border thickness

fname = '/Data/HYP_LR_SR_OB_DR.tif' # downloaded from https://www.naturalearthdata.com/downloads/10m-raster-data/10m-cross-blend-hypso/
ax_all.imshow(imread(fname), origin='upper', extent=[-180, 180, -90, 90])
plt.subplots_adjust(left=0.01, bottom=0.01, right=.99, top=.99)
    
fig.savefig(output_dir+'Pdf/Fig2.pdf', dpi=1200)
fig.savefig(output_dir+'Png/Fig2.png', dpi=1200, transparent=True)

del(fig, ax_all, fname)

### Figure 3

In [12]:
AllTime = input_dir+'ForecastsPatterns/FreqsAll_ERA5.nc'
AllTime = xr.open_dataarray(AllTime)
ZeroTime = input_dir+'ForecastsPatterns/FreqsAll_ERA5_0UTC.nc'
ZeroTime = xr.open_dataarray(ZeroTime)
                             
UTC = pd.Index([False, True], name='ZeroUTC')
FreqsAll = xr.concat([AllTime, ZeroTime], dim=UTC)
FreqsAll = FreqsAll.to_dataframe('Freq').reset_index()
del(AllTime, ZeroTime, UTC)

Freqs = FreqsAll.query('temp_window==0 and ZeroUTC==False') # keep with no temporal window and nly for all hrl data
Freqs = Freqs.pivot_table(columns='cluster', index='temp_subset', values='Freq')*100
Freqs = Freqs.iloc[:-3, :] # don't use statistics for All, and summer-/winter- half periods

In [13]:
# indices for start and end of Summer Half (Summer Half between 16th April - 15th October, inclusive both dates)
Sorted_Dates = np.array(pd.date_range('20040101', '20041231').strftime('%m%d')) # a leap year for getting all dates
Freqs.index = pd.Series(Freqs.index).replace({i_d:i for i, i_d in enumerate(Sorted_Dates)})

Start_Month = [('0'+str(i))[-2:]+'01' for i in range(1, 13)]
start_locs = [np.where(Sorted_Dates==i)[0] for i in Start_Month]
start_locs = np.array(start_locs).flatten()

end_locs = (start_locs-1)[1:]
end_locs = np.insert(end_locs, len(end_locs), len(Sorted_Dates)-1)

x_locs = (start_locs+end_locs)/2
x_ticks = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

In [14]:
fig, ax = plt.subplots(2, 1, gridspec_kw={'height_ratios': [1, .1]}) 
ax = ax.flatten()
Freqs.iloc[:].plot(kind='bar', stacked=True, figsize=(8/2.54, 5/2.54), width=1, color=Cl_cols_used, ax=ax[0])
ax[0].set_ylabel('Frequencies (%)')
ax[0].set_ylim(0, 100)
ax[0].axvline(np.where(Sorted_Dates=='0415')[0]+.5, color='black', linestyle='--', linewidth=1)
ax[0].text(s=16, x=np.where(Sorted_Dates=='0415')[0]+.5, y=0, ha='left', va='bottom', color='white', size=6)
ax[0].axvline(np.where(Sorted_Dates=='1016')[0]-.5, color='black', linestyle='--', linewidth=1)
ax[0].text(s=15, x=np.where(Sorted_Dates=='1016')[0]-.5, y=0, ha='right', va='bottom', color='white', size=6)
h,l = ax[0].get_legend_handles_labels()
ax[0].legend(h, New_names, bbox_to_anchor=(.42, -.1), loc='upper center', ncol=3)
ax[0].yaxis.set_tick_params(width=.25, length=2) # modify y-axis ticks

ax[1].set_ylim(-.5, .5)
for i, j in zip(x_locs, x_ticks): # inside the bars in the Xticks locations state the provided cat_names
    ax[1].text(s=j, x=i, y=0, ha='center', va='center', color='black', size=7)
ax[1].barh(width=len(Freqs), y=0, height=1, color='cyan')
ax[1].barh(width=np.where(Sorted_Dates=='1016')[0]-.5 - (np.where(Sorted_Dates=='0415')[0]+.5), 
           left=np.where(Sorted_Dates=='0415')[0]+.5, y=0, height=1, color='orange')

for i_ax in ax:
    i_ax.set_xlim(-.5, len(Freqs)-.5)
    [i_ax.axvline(i, color='black', linestyle='--', linewidth=0.25) for i in start_locs[1:]]
    
ax[0].get_xaxis().set_visible(False)
ax[1].get_xaxis().set_visible(False)
ax[1].get_yaxis().set_visible(False)
fig.subplots_adjust(left=0.15, bottom=0.25, right=.97, top=.96, wspace=0, hspace=0) # modify margins of figure

fig.savefig(output_dir+'Pdf/Fig3.pdf')
fig.savefig(output_dir+'Png/Fig3.png', dpi=600, facecolor=fig.get_facecolor())

del(FreqsAll, Freqs, Sorted_Dates, Start_Month, start_locs, end_locs, x_locs, x_ticks, fig, ax, h, l, i, j, i_ax)

### Figure 4 (and similar Supplementary ones)

In [15]:
ClustersCondProb = xr.open_dataarray(input_dir+'ForecastsEPEs_Analysis/Connections_Patterns.nc')
ClustersCondProb = ClustersCondProb.sel(longitude=slice(Area_used[1], Area_used[3]), 
                                        latitude=slice(Area_used[0], Area_used[2]))

# create auxiliary data for proper plotting based on pcolormesh
X, Y = np.meshgrid(ClustersCondProb.longitude, ClustersCondProb.latitude)
X_grid2 = np.abs(np.diff(ClustersCondProb.longitude)[0])/2 # when using imshow/pcolor, adjust extend ...
Y_grid2 = np.abs(np.diff(ClustersCondProb.latitude)[0])/2 # ... as the coords refer to the center not edges

'''
It seems that for some reason imshow has a slight shift of the data so the pcolor is used instead.
Nevertheless, pcolor defines each location as lower left corner, and does not plot the last column & row, so some
auxiliary data need to be created to overcome this issue, and use pcolor for generating accurate figures.
Possible explanation of imshow error: https://github.com/matplotlib/matplotlib/issues/12934
''' 
Aux = np.zeros(np.array([len(ClustersCondProb.latitude), len(ClustersCondProb.longitude)])+1) # dimensions of lat/lon

X_all = Aux.copy()
X_all[:, :-1] = X[0, :]
X_all[:, -1] = X.max()+X_grid2*2
X_all -= X_grid2 # shift the data so each location refers to the center and not the edge
Y_all = Aux.copy()
Y_all[1:, :] = Y[:,0][..., np.newaxis]
Y_all[0, :] = Y.max()+Y_grid2*2
Y_all -= Y_grid2 # shift the data so each location refers to the center and not the edge
del(Aux, X_grid2, Y_grid2)

In [16]:
HighestProbValue = ClustersCondProb.max('cluster')
HighestProbValue = (HighestProbValue*100)/(100-HighestProbValue.percentile)
HighestProbCluster = ClustersCondProb.argmax('cluster')

In [17]:
def Fig4_type(perc_used):
    
    Short_names = ['AtlL', 'BscL', 'IbrL', 'SclL', 'BlkL', 'BlSL', 'MedH', 'MnrL', 'MnrH']
    
    fig, ax_all = plt.subplots(2, 3, figsize=(18/2.54, 7.5/2.54), subplot_kw={'projection': ccrs.PlateCarree()})
    ax_all = ax_all.flatten()
    
    temp_subsets = ['All', 'WinterHalf', 'SummerHalf']
    title_aux = list(map(chr, range(97, 123)))[:6] # alphabetic order for subplots naming

    # get aux. data for the cond. prob. plottings
    CondProbMax = HighestProbValue.sel(percentile=perc_used, temporal=temp_subsets).max().values
    CondProbMax = int(np.ceil(CondProbMax))
    Cond_vals_used = np.arange(0, CondProbMax+1)*(100-perc_used)
    Cond_cols_used = sns.color_palette('pink_r', n_colors=len(Cond_vals_used)-1)
    
    # modifications for having correct "BoundaryNorm"
    Cl_vals_used_full = np.insert(Cl_vals_used, len(Cl_vals_used), len(Cl_vals_used))
    
    # plot the best clusters
    for i, j in enumerate(temp_subsets):

        cluster_data = HighestProbCluster.sel(percentile=perc_used, temporal=j).values
        cluster_plot = ax_all[i].pcolor(X_all, Y_all, cluster_data, transform=ccrs.PlateCarree(), 
                                        norm=BoundaryNorm(Cl_vals_used_full, len(Cl_cols_used)), 
                                        cmap=ListedColormap(Cl_cols_used))

        ax_all[i].text(0.5, 1.1, f"{['Year-round', 'WinterHalf', 'SummerHalf'][i]} statistics", fontsize=8.5, 
                       ha='center', transform = ax_all[i].transAxes)

        # add informative colorbar
        Counts = pd.Series(cluster_data.flatten()).value_counts()
        Counts = Counts.reindex(Cl_vals_used).fillna(0)
        Counts = np.round(Counts/cluster_data.size*100, 1).astype(str)
        plot_discrete_bar_inset(col_lims=Cl_vals_used_full, col_used=Cl_cols_used, data_used=Counts, 
                                ax=ax_all[i], brdpad=-2, loc=8, cat_names=Short_names, col_text='black',
                                width='100%', height='15%', xlabel='Percentage of grid cells (%)')


    # plot conditional probabilities of the best 2 clusters
    for i, j in zip([3,4,5], temp_subsets):

        cond_data = HighestProbValue.sel(percentile=perc_used, temporal=j).values
        cond_data = cond_data*(100-perc_used)
        cond_plot = ax_all[i].pcolor(X_all, Y_all, cond_data, transform=ccrs.PlateCarree(),
                                     norm=BoundaryNorm(Cond_vals_used, len(Cond_cols_used)),
                                     cmap=ListedColormap(Cond_cols_used)) 
        
        # add informative colorbar
        cond_values = cond_data.flatten()
        plot_continuous_bar_inset(col_lims=Cond_vals_used, col_used=Cond_cols_used, data=cond_values, ax=ax_all[i], 
                                  width='100%', height='15%', loc=8, brdpad=-2, xlabel='Conditional Probability (%)')

    ax_all[0].text(-.07, .5, 'Cluster of MCP', fontsize=8.5, ha='center', va='center',
                   transform = ax_all[0].transAxes, rotation=90)
    ax_all[3].text(-.07, .5, 'MCP', fontsize=8.5, ha='center', va='center', 
                   transform = ax_all[3].transAxes, rotation=90)

    for i_ax, ax in enumerate(ax_all): # set for all subplots the cartopy map and the extent
        ax.text(0.02, 0.96, f'({title_aux[i_ax]})', fontsize=6, ha='left', va='top', color='black',
                transform = ax_all[i_ax].transAxes, bbox=dict(facecolor='white', edgecolor='none', alpha=0.5, pad=1))
        ax.coastlines(resolution='110m', linewidth=.5, color='black')
        ax.outline_patch.set_linewidth(.2) # reduce the border thickness
        ax.set_extent([X_min, X_max, Y_min, Y_max], crs=ccrs.PlateCarree())
        
    plt.subplots_adjust(left=0.05, bottom=0.12, right=.98, top=.95, hspace=.4, wspace=0.1)
    
    return fig

In [18]:
for i_perc in [90, 95, 99]:
    Fig4 = Fig4_type(perc_used=i_perc)
    Fig4.savefig(output_dir+f'Pdf/Fig4_{i_perc}.pdf')
    Fig4.savefig(output_dir+f'Png/Fig4_{i_perc}.png', dpi=600, facecolor=Fig4.get_facecolor())
    if i_perc!=95: plt.close()
    
del(i_perc, Fig4, ClustersCondProb, X, Y, X_all, Y_all, HighestProbValue, HighestProbCluster, Fig4_type)

### Figure 5 (and similar Supplementary ones)

In [19]:
BS_all = xr.open_dataset(input_dir+'ForecastsEPEs_Analysis/BS_ERA5_All.nc')
BS_all = BS_all.sel(longitude=slice(Area_used[1], Area_used[3]), latitude=slice(Area_used[0], Area_used[2]))

BSS_all = BS_all['BSS']
BSS_all = BSS_all.where(BSS_all>0)

# create auxiliary data for proper plotting based on pcolormesh
X, Y = np.meshgrid(BS_all.longitude, BS_all.latitude)
X_grid2 = np.abs(np.diff(BS_all.longitude)[0])/2 # when using imshow/pcolor, adjust extend as the coords ...
Y_grid2 = np.abs(np.diff(BS_all.latitude)[0])/2 # ... refer to the center of each cell not its edges

'''
It seems that for some reason imshow has a slight shift of the data so the pcolor is used instead.
Nevertheless, pcolor defines each location as lower left corner, and does not plot the last column & row, so some
auxiliary data need to be created to overcome this issue, and use pcolor for generating accurate figures.
Possible explanation of imshow error: https://github.com/matplotlib/matplotlib/issues/12934
''' 
Aux = np.zeros(np.array(BSS_all.shape[-2:])+1) # dimensions of lat/lon

X_all = Aux.copy()
X_all[:, :-1] = X[0, :]
X_all[:, -1] = X.max()+X_grid2*2
X_all -= X_grid2 # shift the data so each location refers to the center and not the edge
Y_all = Aux.copy()
Y_all[1:, :] = Y[:,0][..., np.newaxis]
Y_all[0, :] = Y.max()+Y_grid2*2
Y_all -= Y_grid2 # shift the data so each location refers to the center and not the edge
del(Aux, X_grid2, Y_grid2)

In [20]:
def Fig5_type(perc_used=95):
    
    Plot_used = BSS_all.sel(percentile=perc_used, Method=['All_Patt', 'HalfYear_Patt'])
    
    Plot_used = Plot_used*100 # make the results as %
    Max_value = Plot_used.max().values
    Max_value = np.ceil(Max_value)

    Cols_used = sns.color_palette('viridis', n_colors=10)
    Vals_used = np.linspace(0, Max_value, 11)
    
    fig, ax_all = plt.subplots(1, 2, figsize=(18/2.54, 6/2.54), subplot_kw={'projection': ccrs.PlateCarree()})
    ax_all = ax_all.flatten()
    for i, ax in enumerate(ax_all): # set for all subplots the cartopy map and the extent
        ax.coastlines(resolution='110m', linewidth=.5, color='black')
        ax.outline_patch.set_linewidth(.2) # reduce the border thickness
        ax.set_extent([X_min, X_max, Y_min, Y_max], crs=ccrs.PlateCarree())
        
        data_mesh = Plot_used.isel(Method=i).values
        bss_plot = ax.pcolor(X_all, Y_all, data_mesh, transform=ccrs.PlateCarree(), 
                             norm=BoundaryNorm(Vals_used, len(Cols_used)), cmap=ListedColormap(Cols_used))

        OutperformsLocs = np.isnan(data_mesh).sum()
        OutperformsLocs = (data_mesh.size-OutperformsLocs)/data_mesh.size*100 # number of significant locations
        OutperformsLocs = np.round(OutperformsLocs, 2) # % locs that cluster is significant
        
        i_title = ['(a) Year-round Cond. Prob.', '(b) Half-year Cond. Prob.'][i]       
        ax.set_title('{}: outperforms in {}% of cells'.format(i_title, OutperformsLocs), 
                     pad=4, size=8, loc='left')

        # add informative colorbar
        values_used = data_mesh.flatten()[~np.isnan(data_mesh.flatten())] # only outperforming values
        plot_continuous_bar_inset(col_lims=Vals_used, col_used=Cols_used, data=values_used, ax=ax, 
                                  width='100%', height='15%', loc=8, brdpad=-3, xlabel='BSS (%)')

    plt.subplots_adjust(left=0.02, bottom=0.11, right=.98, top=1, hspace=.0, wspace=0.1)

    return fig

In [21]:
for i_perc in [90, 95, 99]:
    Fig5 = Fig5_type(perc_used=i_perc)
    Fig5.savefig(output_dir+f'Pdf/Fig5_{i_perc}.pdf')
    Fig5.savefig(output_dir+f'Png/Fig5_{i_perc}.png', dpi=600, facecolor=Fig5.get_facecolor())
    if i_perc!=95: plt.close()
    
del(i_perc, Fig5, BS_all, BSS_all, X, Y, X_all, Y_all, Fig5_type)

### Figure 6

In [22]:
AllTime = input_dir+'ForecastsPatterns/BS_Clusters.nc'
AllTime = xr.open_dataset(AllTime)
ZeroTime = input_dir+'ForecastsPatterns/BS_Clusters_0UTC.nc'
ZeroTime = xr.open_dataset(ZeroTime)

UTC = pd.Index([False, True], name='ZeroUTC')
BS_PatternsAll = xr.concat([AllTime, ZeroTime], dim=UTC)

# add np.nan for 0 lead-time, else pointplot shifts data cause it is for categorical so does not know actual location
BS_PatternsAll_aux = BS_PatternsAll.sel(Lead_days=1)*np.nan
BS_PatternsAll_aux = BS_PatternsAll_aux.assign_coords({'Lead_days': 0})
BS_PatternsAll = xr.concat([BS_PatternsAll_aux, BS_PatternsAll], dim='Lead_days')

BS_PatternsAll = BS_PatternsAll.to_dataframe().reset_index()
BS_PatternsAll['BSS_Masked'] = BS_PatternsAll.BSS.mask(BS_PatternsAll.Sign<=.90) # 90% CI for sign on BSS
BS_PatternsAll = BS_PatternsAll.query("Method=='Cluster' and Flex_win==0 and bootstrap=='P50'")
del(UTC, AllTime, ZeroTime, BS_PatternsAll_aux)

In [23]:
def Fig6_type(zero_utc=False):
    
    BS_Patterns = BS_PatternsAll.query('ZeroUTC==@zero_utc')
    
    Y_max = np.ceil(BS_Patterns.BSS.max()*10)/10
    Y_min = np.floor(BS_Patterns.BSS.min()*10)/10
    Markers = ['o', 'v', '^', '<', '>', 's', 'p', '*', 'X']
    fig, ax_all = plt.subplots(2, 2, figsize=(18/2.54, 11/2.54))
    ax_all = ax_all.flatten()
    for i, i_seas in zip([0, 2, 3], ['All', 'WinterHalf', 'SummerHalf']):
        sns.lineplot(data=BS_Patterns.query('Season==@i_seas'), x='Lead_days', y='BSS', 
                     hue='Cluster', linewidth=3, palette=Cl_cols_used, alpha=.15, ax=ax_all[i])
        sns.pointplot(data=BS_Patterns.query('Season==@i_seas'), x='Lead_days', y='BSS_Masked',
                      hue='Cluster', linewith=4, palette=Cl_cols_used, alpha=1, markers=Markers, ax=ax_all[i])
        [i_col.set_sizes([20]) for i_col in ax_all[i].collections]
        
        ax_all[i].axhline(0, linestyle='--', color='black')
        ax_all[i].legend().remove()
        ax_all[i].set_xlim(0, 27)
        ax_all[i].set_xticks(np.arange(0, 28, 3))
        ax_all[i].set_xticklabels(np.arange(0, 28, 3))
        
        ax_all[i].set_xlabel('Lead days')
        ax_all[i].set_ylim(Y_min, Y_max)
        ax_all[i].set_ylabel('BSS')
        if i_seas == 'All': i_seas = 'Year-round'
        ax_all[i].set_title(f"({['a', 'a', 'b', 'c'][i]}) {i_seas}", pad=4, loc='left', size=9)
        ax_all[i].xaxis.set_tick_params(width=.25, length=2) # modify x-axis ticks
        ax_all[i].yaxis.set_tick_params(width=.25, length=2) # modify y-axis ticks

    # plot only legend on the upper right plot
    mark = []
    for i in range(len(New_names)):
        mark_i,  = ax_all[1].plot([1, 3], [2, 4], lw=2, marker=Markers[i], color=Cl_cols_used[i], alpha=0)
        mark.append(mark_i)

    leg = ax_all[1].legend(mark, New_names, loc=10)  
    fr = leg.get_frame() # get legend frame
    fr.set_lw(0) # modify frame's thickness
    for i, l in enumerate(leg.get_lines()):
        l.set_alpha(1)
        l.set_marker(Markers[i])
    [ ax_all[1].spines[i].set_visible(False) for i in ['top', 'right', 'bottom', 'left'] ]
    ax_all[1].set_xticks([])
    ax_all[1].set_yticks([]) 

    plt.subplots_adjust(left=0.1, bottom=0.1, right=.99, top=.95, wspace=0.25, hspace=0.45)
    return fig

In [24]:
for i_zero_utc in [False, True]:
    Fig6 = Fig6_type(zero_utc=i_zero_utc)
    Fig6.savefig(output_dir+f'Pdf/Fig6_0UTC_{i_zero_utc}.pdf')
    Fig6.savefig(output_dir+f'Png/Fig6_0UTC_{i_zero_utc}.png', dpi=600, facecolor=Fig6.get_facecolor())
    if i_zero_utc!=False: plt.close()
    
del(i_zero_utc, Fig6, Fig6_type, BS_PatternsAll)

### Figure S5

In [25]:
AllTime = input_dir+'ForecastsPatterns/Decomp_Aggr.nc'
AllTime = xr.open_dataset(AllTime)
ZeroTime = input_dir+'ForecastsPatterns/Decomp_Aggr_0UTC.nc'
ZeroTime = xr.open_dataset(ZeroTime)

UTC = pd.Index([False, True], name='ZeroUTC')
DecompositionAll = xr.concat([AllTime, ZeroTime], dim=UTC).to_array()

DecompositionAll = DecompositionAll.sel(Flex_win=0).reset_coords(drop=True)
DecompositionAll = DecompositionAll.to_dataframe('Value').reset_index()
del(UTC, AllTime, ZeroTime)

In [26]:
def FigS5_type(zero_utc=False):

    Decomposition = DecompositionAll.query('ZeroUTC==@zero_utc')
    
    fig, ax_all = plt.subplots(3, 3, figsize=(18/2.54, 12.5/2.54))
    ax_all = ax_all.flatten()
    for i, i_season in enumerate(['All', 'WinterHalf', 'SummerHalf']):
        i_p = sns.lineplot(data=Decomposition.query('bootstrap=="P50" and Season==@i_season and Stat=="Res."'), 
                           alpha=.6, x='Lead_days', y='Value', hue='Cluster', palette=Cl_cols_used, 
                           linewidth=2, ax=ax_all[i])


    for i, i_season in enumerate(['All', 'WinterHalf', 'SummerHalf']):
        i_p = sns.lineplot(data=Decomposition.query('bootstrap=="P50" and Season==@i_season and Stat=="Rel."'), 
                           alpha=.6, x='Lead_days', y='Value', hue='Cluster', palette=Cl_cols_used, 
                           linewidth=2, ax=ax_all[i+3])

    for i, i_season in enumerate(['All', 'WinterHalf', 'SummerHalf']):
        i_p = sns.lineplot(data=Decomposition.query('bootstrap=="P50" and Season==@i_season and Stat=="Unc."'), 
                           alpha=.6, x='Lead_days', y='Value', hue='Cluster', palette=Cl_cols_used, 
                           linewidth=2, ax=ax_all[i+6])

    for i, ax in enumerate(ax_all):
        if i == 0: h, l = ax.get_legend_handles_labels()
        ax.legend().remove()
        ax.set_xlim(0, 45)
        ax.set_xticks(np.arange(0, 46, 5))
        ax.set_ylabel('')
        ax.set_xlabel('Lead days')
        ax.xaxis.set_tick_params(width=.25, length=2) # modify x-axis ticks
        ax.yaxis.set_tick_params(width=.25, length=2) # modify y-axis ticks

        ax.text(0.02, 0.98, f"({['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i'][i]})", fontsize=6, ha='left', va='top',
                  color='black', transform = ax.transAxes, 
                  bbox=dict(facecolor='white', edgecolor='none', alpha=0.5, pad=1))
        if i <= 2:
            ax.text(0.5, 1.1, ['Year-round', 'WinterHalf', 'SummerHalf'][i], fontsize=8, 
                    ha='center', va='center', transform = ax.transAxes)

        if i in [0, 3, 6]:
            ax.text(-.3, .5, ['Resolution', '', '', 'Reliability', '', '', 'Uncertainty'][i], fontsize=8, 
                    ha='center', va='center', transform = ax.transAxes, rotation=90)

    [i_h.set_linewidth(2) for i_h in h]
    l = New_names  
    fig.legend(h, l, ncol=len(New_names)//2+1, loc='lower center', bbox_to_anchor=[0.5, 0.0])
    plt.subplots_adjust(left=0.09, bottom=0.18, right=.99, top=.93, wspace=0.25, hspace=0.4)
    
    return fig

In [27]:
for i_zero_utc in [False, True]:
    FigS5 = FigS5_type(zero_utc=i_zero_utc)
    FigS5.savefig(output_dir+f'Pdf/FigS5_0UTC_{i_zero_utc}.pdf')
    FigS5.savefig(output_dir+f'Png/FigS5_0UTC_{i_zero_utc}.png', dpi=600, facecolor=FigS5.get_facecolor())
    if i_zero_utc!=False: plt.close()
    
del(i_zero_utc, FigS5, FigS5_type, DecompositionAll)

### Figure 7

In [28]:
AllTime = input_dir+'ForecastsPatterns/FreqOccur_Forecasts.nc'
AllTime = xr.open_dataset(AllTime)
ZeroTime = input_dir+'ForecastsPatterns/FreqOccur_Forecasts_0UTC.nc'
ZeroTime = xr.open_dataset(ZeroTime)

UTC = pd.Index([False, True], name='ZeroUTC')
FreqBiasesAll = xr.concat([AllTime, ZeroTime], dim=UTC)

# add np.nan for 0 lead-time, else pointplot shifts data cause it is for categorical so does not know actual location
FreqBiasesAll_aux = FreqBiasesAll.sel(Lead_days=1)*np.nan
FreqBiasesAll_aux = FreqBiasesAll_aux.assign_coords({'Lead_days': 0})
FreqBiasesAll = xr.concat([FreqBiasesAll_aux, FreqBiasesAll], dim='Lead_days')

# add difference of biases when using all timesteps for deriving the ERA5 patterns, and only 0UTC timesteps
FreqBiasesAllDifs = np.abs(FreqBiasesAll.sel(ZeroUTC=False)) - np.abs(FreqBiasesAll.sel(ZeroUTC=True))
FreqBiasesAllDifs = FreqBiasesAllDifs.assign_coords({'ZeroUTC': 'Dif'})
FreqBiasesAll = xr.concat([FreqBiasesAll, FreqBiasesAllDifs], dim='ZeroUTC')

FreqBiasesAll = FreqBiasesAll.sel(Flex_win=0, Member='Reforecasts', bootstrap='BS_Median')
FreqBiasesAll = FreqBiasesAll.to_dataframe().reset_index()

FreqBiasesAll['FreqBiasMasked'] = FreqBiasesAll.FreqBias
FreqBiasesAll.loc[FreqBiasesAll.SignDeviations==False, 'FreqBiasMasked'] = np.nan
FreqBiasesAll.Cluster = FreqBiasesAll.Cluster.map({i:j for i, j in enumerate(New_names)})
del(UTC, AllTime, ZeroTime, FreqBiasesAll_aux, FreqBiasesAllDifs)

In [29]:
def Fig7_type(zero_utc=False):
    
    FreqBiases = FreqBiasesAll.query('ZeroUTC==@zero_utc')
    Y_max = np.ceil(np.abs(FreqBiases.FreqBias).max()/10)*10
    Markers = ['o', 'v', '^', '<', '>', 's', 'p', '*', 'X']
    fig, ax_all = plt.subplots(2, 2, figsize=(18/2.54, 11/2.54))
    ax_all = ax_all.flatten()
    for i, i_seas in zip([0, 2, 3], ['All', 'WinterHalf', 'SummerHalf']):
        sns.lineplot(data=FreqBiases.query('Season==@i_seas'), x='Lead_days', y='FreqBias', hue='Cluster',
                     linewidth=2, palette=Cl_cols_used, alpha=.15, ax=ax_all[i])
        sns.pointplot(data=FreqBiases.query('Season==@i_seas'), x='Lead_days', y='FreqBiasMasked', hue='Cluster',
                      scale=1.5, palette=Cl_cols_used, alpha=1, markers=Markers, ax=ax_all[i])
        [i_col.set_sizes([20]) for i_col in ax_all[i].collections]
        ax_all[i].axhline(0, linestyle='--', color='black')
        ax_all[i].legend().remove()
        ax_all[i].set_xlim(0, 45)
        ax_all[i].set_xticks(np.arange(0, 46, 5))
        ax_all[i].set_xticklabels(np.arange(0, 46, 5))
        ax_all[i].set_xlabel('Lead days')
        ax_all[i].set_ylim(-Y_max, Y_max)
        ax_all[i].set_ylabel('Relative Bias (%)')
        if i_seas == 'All': i_seas = 'Year-round'
        ax_all[i].set_title(f"({['a', 'a', 'b', 'c'][i]}) {i_seas}", pad=4, loc='left', size=9)
        ax_all[i].xaxis.set_tick_params(width=.25, length=2) # modify x-axis ticks
        ax_all[i].yaxis.set_tick_params(width=.25, length=2) # modify y-axis ticks

    # plot only legend on the upper right plot
    mark = []
    for i in range(len(New_names)):
        mark_i,  = ax_all[1].plot([1, 3], [2, 4], lw=2, marker=Markers[i], color=Cl_cols_used[i], alpha=0)
        mark.append(mark_i)

    leg = ax_all[1].legend(mark, New_names, loc=10)  
    fr = leg.get_frame() # get legend frame
    fr.set_lw(0) # modify frame's thickness
    for i, l in enumerate(leg.get_lines()):
        l.set_alpha(1)
        l.set_marker(Markers[i])
    [ ax_all[1].spines[i].set_visible(False) for i in ['top', 'right', 'bottom', 'left'] ]
    ax_all[1].set_xticks([])
    ax_all[1].set_yticks([]) 

    plt.subplots_adjust(left=0.1, bottom=0.1, right=.99, top=.95, wspace=0.25, hspace=0.45)
    
    return fig

In [30]:
for i_zero_utc in [False, True, 'Dif']:
    Fig7 = Fig7_type(zero_utc=i_zero_utc)
    Fig7.savefig(output_dir+f'Pdf/Fig7_0UTC_{i_zero_utc}.pdf')
    Fig7.savefig(output_dir+f'Png/Fig7_0UTC_{i_zero_utc}.png', dpi=600, facecolor=Fig7.get_facecolor())
    if i_zero_utc!=False: plt.close()
    
del(i_zero_utc, Fig7, Fig7_type, FreqBiasesAll)

### Figure 8 & Figure 9

In [31]:
# batlow colorbar from https://www.fabiocrameri.ch/batlow/
btlw = ['#011959', '#103F60', '#1C5A62', '#3C6D56', '#687B3E', '#9D892B', '#D29343', '#F8A17B', '#FDB7BC', '#FACCFA']
Cols_used = btlw[::-1][2:]
Vals_used_general = np.array([0, 3, 5, 7, 9, 11, 13, 15, 16])

In [32]:
BS_EPEs = xr.open_dataset(input_dir+f'ForecastsEPEs_Analysis/BS_leaddays.nc')
Weights = np.cos(np.radians(BS_EPEs.latitude))

In [33]:
# create auxiliary data for proper plotting based on pcolormesh
X, Y = np.meshgrid(BS_EPEs.longitude, BS_EPEs.latitude)
X_grid2 = np.abs(np.diff(BS_EPEs.longitude)[0])/2 # when using imshow/pcolor, adjust extend as the coords ...
Y_grid2 = np.abs(np.diff(BS_EPEs.latitude)[0])/2 # ... refer to the center of each cell not its edges

'''
It seems that for some reason imshow has a slight shift of the data so the pcolor is used instead.
Nevertheless, pcolor defines each location as lower left corner, and does not plot the last column & row, so some
auxiliary data need to be created to overcome this issue, and use pcolor for generating accurate figures.
Possible explanation of imshow error: https://github.com/matplotlib/matplotlib/issues/12934
''' 
Aux = np.zeros(np.array([len(BS_EPEs['BS'].latitude), len(BS_EPEs['BS'].longitude)])+1) # dimensions of lat/lon

X_all = Aux.copy()
X_all[:, :-1] = X[0, :]
X_all[:, -1] = X.max()+X_grid2*2
X_all -= X_grid2 # shift the data so each location refers to the center and not the edge
Y_all = Aux.copy()
Y_all[1:, :] = Y[:,0][..., np.newaxis]
Y_all[0, :] = Y.max()+Y_grid2*2
Y_all -= Y_grid2 # shift the data so each location refers to the center and not the edge
del(Aux, X_grid2, Y_grid2)

In [34]:
def aggregated_stats(input_data, input_coords):
    x_min, x_max, y_min, y_max = input_coords
    data = input_data.sel(latitude=slice(y_max, y_min), longitude=slice(x_min, x_max))
    data = data.weighted(Weights).mean(['latitude', 'longitude'])
    
    return data

In [35]:
AggregatedData = [aggregated_stats(BS_EPEs, i_val) for i_val in Areas.values()]
AggregatedData = xr.concat(AggregatedData, dim=pd.Index(list(Areas.keys()), name='Area'))
Aggregated = AggregatedData.sel(bootstraps_frcst='P50').to_dataframe().reset_index()
Aggregated = Aggregated.query('Method in ["EPEs_Direct", "HalfYear_Patt"]')

In [36]:
def Fig8_type(perc_used=95):
    
    Y_max = np.ceil(Aggregated.BSS.max()*10)/10
    Y_min = np.floor(Aggregated.BSS.min()*10)/10
    
    fig, ax_all = plt.subplots(2, 3, figsize=(18/2.54, 9/2.54))
    ax_all = ax_all.flatten()
    
    for i, i_area in enumerate(Areas):
        sns.lineplot(data=Aggregated.query('percentile==@perc_used and Area==@i_area'), x='leaddays', y='BSS', 
                     hue='Method', linewidth=2, alpha=.3, ax=ax_all[i], hue_order=Aggregated.Method.unique())
        sns.lineplot(data=Aggregated.query('percentile==@perc_used and Area==@i_area and BSS>0'), x='leaddays', 
                     y='BSS', hue='Method', linewidth=3, ax=ax_all[i], hue_order=Aggregated.Method.unique())
        ax_all[i].axhline(0, linestyle='--', color='black')
        if i!=0: ax_all[i].legend().remove()

        ax_all[i].set_xlabel('Lead days')
        ax_all[i].set_ylim(Y_min, Y_max)
        ax_all[i].set_ylabel('BSS')
        ax_all[i].set_title(f"({['a', 'b', 'c', 'd', 'e', 'f'][i]}) {i_area}", pad=4, loc='left', size=8)
        ax_all[i].xaxis.set_tick_params(width=.25, length=2) # modify x-axis ticks
        ax_all[i].yaxis.set_tick_params(width=.25, length=2) # modify y-axis ticks
        ax_all[i].set_xlim(0, 45)
        ax_all[i].set_xticks(np.arange(0, 46, 5))
        ax_all[i].set_xticklabels(np.arange(0, 46, 5))
        
        if i>0:
            ax_sub = inset_axes(ax_all[i], loc=1, width='60%', height='40%', #bbox_to_anchor=(0, 1, 1, 1),
                                axes_class=cartopy.mpl.geoaxes.GeoAxes, 
                                axes_kwargs=dict(map_projection=ccrs.PlateCarree()))
            ax_sub.coastlines(resolution='110m', linewidth=.5, color='black')
            ax_sub.set_extent([X.min(), X.max(), Y.min(), Y.max()], crs=ccrs.PlateCarree())
            x_min, x_max, y_min, y_max = Areas[i_area]
            rect = plt.Rectangle((x_min, y_min),x_max-x_min, y_max-y_min, fill=False, color='red', linewidth=.75) 
            ax_sub.add_patch(rect) # add rectangle with the actual area used for the Precipitation analysis

    # plot only the legend
    sns.lineplot(data=Aggregated.query('percentile==@perc_used and Area==@i_area'), x='leaddays', y='BSS', 
                 hue='Method', linewidth=2, alpha=0, ax=ax_all[0], hue_order=Aggregated.Method.unique())
    ax_all[0].set_xlabel('Lead days')
    ax_all[0].set_ylim(Y_min, Y_max)
        
    h, l = ax_all[0].get_legend_handles_labels()
    l = ['Direct', 'Indirect']
    leg = ax_all[0].legend(h, l)
    [line.set_linewidth(2) for line in leg.get_lines()]
    
    plt.subplots_adjust(left=0.07, bottom=0.12, right=.99, top=.93, wspace=0.3, hspace=0.5)
    return fig

In [37]:
for i_perc in AggregatedData.percentile.values:
    Fig8 = Fig8_type(perc_used=i_perc)
    Fig8.savefig(output_dir+f'Pdf/Fig8_{i_perc}.pdf')
    Fig8.savefig(output_dir+f'Png/Fig8_{i_perc}.png', dpi=600, facecolor=Fig8.get_facecolor())
    if i_perc!=95: plt.close()
    
del(i_perc, Fig8, Fig8_type)

In [38]:
# https://stackoverflow.com/a/33893692/ @Divakar
# https://stackoverflow.com/questions/61339393/numpy-forward-fill-with-condition
def numpy_binary_closing(mask,W_gap):
    
    W = W_gap+1
    # Define kernel
    K = np.ones(W)

    # Perform dilation and threshold at 1
    dil = np.convolve(mask,K)>=1

    # Perform erosion on the dilated mask array and threshold at given threshold
    dil_erd = np.convolve(dil,K)>= W
    return dil_erd[W-1:-W+1]

def ffill_windowed(a, W_gap):
    if W_gap>0:
        mask = ~np.isnan(a)
        mask_ext = numpy_binary_closing(mask,W_gap)

        p = mask_ext & ~mask
        idx = np.maximum.accumulate(mask*np.arange(len(mask)))
        out = a.copy()
        out[p] = out[idx[p]]
    else:
        out = a
    return out

In [39]:
# get a mask for the indirect forecast, based on the boostrap data of perfect knowledge of the patterns
BSS_Ref_Mask_PerfectForecast = BS_EPEs['Sign_Ref'].sel(Method='HalfYear_Patt_Perfect').mean('leaddays')>.9 
BSS_Ref_Mask_PerfectForecast = BSS_Ref_Mask_PerfectForecast.reset_coords(drop=True)

# get initial data where forecasts beat reference
BSS_Ref = BS_EPEs['Sign_Ref'].sel(Method=['EPEs_Direct', 'HalfYear_Patt'])>0.9
BSS_Ref = BSS_Ref.where(BSS_Ref)

# for indirect forecasts mask also based on the boostrapping of perfect knowledge of patterns
BSS_Ref_Indir = BSS_Ref.sel(Method='HalfYear_Patt').where(BSS_Ref_Mask_PerfectForecast).reset_coords(drop=True)
BSS_Ref_Dir = BSS_Ref.sel(Method='EPEs_Direct').reset_coords(drop=True)
BSS_Ref_All = xr.concat([BSS_Ref_Dir, BSS_Ref_Indir], dim=pd.Index(['EPEs_Direct', 'EPEs_Indirect'], name='Method'))

# get last lead day that forecasts (indirect and direct) beat reference (allow only 1 non-sign. gap between the steps)
BSS_Ref_All_values = np.apply_along_axis(ffill_windowed, 1, BSS_Ref_All, 1)
BSS_Ref_All[:] = BSS_Ref_All_values
BSS_Ref_All = BSS_Ref_All.sel(leaddays=range(2, BSS_Ref_All.leaddays.max().values+1)) # don't use lead 1, in case ...
BSS_Ref_All = BSS_Ref_All.cumsum('leaddays', skipna=False) # ... is np.nan (1 np.nan gap is allowed so this is fine)
BSS_Ref = BSS_Ref_All.max('leaddays') + 1 # add 1, since the LeadDays start from 2nd day and not the 1st one

# get locations where Indirect BSS has extended horizon compared to Direct BSS
BSS_Ind_Max = BSS_Ref.sel(Method='EPEs_Indirect')
BSS_Ind_Max = BSS_Ind_Max.where(BSS_Ind_Max>BSS_Ref.sel(Method='EPEs_Direct')).reset_coords(drop=True)

# get first day that indirect forecasts beat the direct ones and the climatology
BSS_Combo = BS_EPEs['Sign_Combo'].sel(Method='HalfYear_Patt')>0.9
BSS_Combo = BSS_Combo.where(BSS_Combo & BSS_Ref_Mask_PerfectForecast)*BSS_Combo.leaddays 
BSS_Combo = BSS_Combo.min('leaddays').reset_coords(drop=True)
BSS_Combo = BSS_Combo.where(BSS_Combo<=BSS_Ref.sel(Method='EPEs_Indirect').reset_coords(drop=True))
BSS_Combo = xr.concat([BSS_Ind_Max, BSS_Combo], dim='aux').min('aux') # needed as for clim. sign. 1 nan gap allowed

del(BSS_Ref_Mask_PerfectForecast, BSS_Ref_Indir, BSS_Ref_Dir, BSS_Ref_All, BSS_Ref_All_values, BSS_Ind_Max)

In [40]:
def Fig9_type(perc_used=95):
    
    BSS_horizon = BSS_Ref.sel(percentile=perc_used).reset_coords(drop=True)
    Vals_used = np.array(list(Vals_used_general)) # this is needed, otherwise Vals_used_general changes as well
    max_lead = int(BSS_horizon.max().values)
    if max_lead > Vals_used[-1]: Vals_used[-1] = max_lead
    
    fig, ax_all = plt.subplots(2, 2, figsize=(18/2.54, 12/2.54), subplot_kw={'projection': ccrs.PlateCarree()})
    ax_all = ax_all.flatten()
    for i, ax in enumerate(ax_all[:2]):

        data_mesh = BSS_horizon.isel(Method=i).values
        ax.pcolor(X_all, Y_all, data_mesh, transform=ccrs.PlateCarree(), 
                  norm=BoundaryNorm(Vals_used, len(Cols_used)), cmap=ListedColormap(Cols_used)) 

        aux_title = ['direct', 'indirect']
        ax.set_title(f"({['a', 'b'][i]}) Forecasting horizon (in days) with {aux_title[i]} forecasts", 
                     pad=4, size=8, loc='left')

        # get the values so that the statistics are created and plotted on the colorbar
        AllData = data_mesh.flatten()[~np.isnan(data_mesh.flatten())]     
        Counts = pd.DataFrame({'Bins': [f'{i}~{j-1}' for i, j in zip(Vals_used[:-1], Vals_used[1:])], 
                               'Percentages': np.nan})
        Counts.iloc[-1, 0] = f'{Vals_used[-2]}~{Vals_used[-1]}'

        for i_bin in range(len(Counts)):
            if i_bin==len(Counts)-1:
                i_counts = sum((AllData >= Vals_used[i_bin]) & (AllData <= Vals_used[i_bin+1]))
            else:
                i_counts = sum((AllData >= Vals_used[i_bin]) & (AllData < Vals_used[i_bin+1]))
            Counts.iloc[i_bin, 1] = np.round(i_counts/len(data_mesh.flatten())*100, 2).astype(str)

        ax_sub = plot_discrete_bar_inset(col_lims=np.arange(len(Vals_used)), col_used=Cols_used, col_text='white',
                                         data_used=Counts.Percentages.values, loc=8, width='100%', height='15%',
                                         brdpad=-3, cat_names=Counts.Bins.values, bar_text_size=6.5, ax=ax,
                                         xlabel='Percentage of grid cells (%)')

    HorizonDif = BSS_horizon.fillna(0).diff('Method').values[0, ...]
    MaxDif = int(np.abs(HorizonDif).max())
    Colors_used = sns.color_palette('YlOrRd_r', n_colors=4) + ['white'] + sns.color_palette('GnBu', n_colors=4)
    Colors_limits = [-10, -6, -4, -2, 0, 1, 3, 5, 7, 10]
    if MaxDif>max(Colors_limits):
        Colors_limits[0] = -MaxDif
        Colors_limits[-1] = MaxDif

    ax_all[2].pcolor(X_all, Y_all, HorizonDif, transform=ccrs.PlateCarree(), 
                     norm=BoundaryNorm(Colors_limits, len(Colors_used)), cmap=ListedColormap(Colors_used)) 
    ax_all[2].set_title('(c) Indirect-direct forecasting horizon difference (in days)', pad=4, size=8, loc='left')

    AllData = HorizonDif.flatten()  
    Counts = pd.DataFrame({'Bins': [f'{i}~{j-1}' for i, j in zip(Colors_limits[:-1], Colors_limits[1:])], 
                           'Percentages': np.nan})
    Counts.iloc[-1, 0] = f'{Colors_limits[-2]}~{Colors_limits[-1]}'
    Counts.iloc[4,0] = '0'

    for i_bin in range(len(Counts)):
        if i_bin==len(Counts)-1:
            i_counts = sum((AllData >= Colors_limits[i_bin]) & (AllData <= Colors_limits[i_bin+1]))
        else:
            i_counts = sum((AllData >= Colors_limits[i_bin]) & (AllData < Colors_limits[i_bin+1]))
        Counts.iloc[i_bin, 1] = np.round(i_counts/len(data_mesh.flatten())*100, 2).astype(str)

    ax_sub = plot_discrete_bar_inset(col_lims=np.arange(len(Colors_limits)), col_used=Colors_used, col_text='black',  
                                     data_used=Counts.Percentages.values, loc=8, width='100%', height='15%', 
                                     brdpad=-3, cat_names=Counts.Bins.values, bar_text_size=6.5, ax=ax_all[2],
                                     xlabel='Percentage of grid cells (%)')

    MinFrcst = BSS_Combo.sel(percentile=perc_used).values
    MinFrcst_max = int(MinFrcst[~np.isnan(MinFrcst)].max())
    Colors_limits = np.array([0, 6, 8, 10, 12, 15]) # np.arange(0, 15, 2)
    Colors_used = sns.color_palette('Purples_r', n_colors=len(Colors_limits))[:-1]
    if MinFrcst_max>max(Colors_limits):
        Colors_limits[-1] = MinFrcst_max

    ax_all[3].pcolor(X_all, Y_all, MinFrcst, transform=ccrs.PlateCarree(), 
                     norm=BoundaryNorm(Colors_limits, len(Colors_used)), cmap=ListedColormap(Colors_used)) 
    ax_all[3].set_title('(d) First lead day when indirect forecasts beat direct one', pad=4, size=8, loc='left')

    AllData = MinFrcst.flatten()[~np.isnan(MinFrcst.flatten())]  
    Counts = pd.DataFrame({'Bins': [f'{i}~{j-1}' for i, j in zip(Colors_limits[:-1], Colors_limits[1:])], 
                           'Percentages': np.nan})
    Counts.iloc[-1, 0] = f'{Colors_limits[-2]}~{Colors_limits[-1]}'

    for i_bin in range(len(Counts)):
        if i_bin==len(Counts)-1:
            i_counts = sum((AllData >= Colors_limits[i_bin]) & (AllData <= Colors_limits[i_bin+1]))
        else:
            i_counts = sum((AllData >= Colors_limits[i_bin]) & (AllData < Colors_limits[i_bin+1]))
        Counts.iloc[i_bin, 1] = np.round(i_counts/MinFrcst.size*100, 2).astype(str)

    ax_sub = plot_discrete_bar_inset(col_lims=np.arange(len(Colors_limits)), col_used=Colors_used, col_text='black',  
                                     data_used=Counts.Percentages.values, loc=8, width='100%', height='15%', 
                                     brdpad=-3, cat_names=Counts.Bins.values, bar_text_size=6.5, ax=ax_all[3],
                                     xlabel='Percentage of grid cells (%)')

    for ax in ax_all:
        ax.coastlines(resolution='110m', linewidth=.75, color='black')
        ax.set_extent([X_min, X_max, Y_min, Y_max], crs=ccrs.PlateCarree())

    plt.subplots_adjust(left=0.02, bottom=0.05, right=.98, top=1, wspace=0.05, hspace=0.05)
    
    return fig

In [41]:
for i_perc in [90, 95, 99]:
    Fig9 = Fig9_type(perc_used=i_perc)
    Fig9.savefig(output_dir+f'Pdf/Fig9_{i_perc}.pdf')
    Fig9.savefig(output_dir+f'Png/Fig9_{i_perc}.png', dpi=600, facecolor=Fig9.get_facecolor())
    if i_perc!=95: plt.close()
    
del(i_perc, Fig9, Fig9_type)

In [42]:
def GraphicalAbstract(perc_used=95):
    
    BSS_horizon = BSS_Ref.sel(percentile=perc_used).reset_coords(drop=True)
    Vals_used = np.array(list(Vals_used_general)) # this is needed, otherwise Vals_used_general changes as well
    max_lead = int(BSS_horizon.max().values)
    if max_lead > Vals_used[-1]: Vals_used[-1] = max_lead
    
    fig, ax_all = plt.subplots(2, 1, figsize=(5/2.54, 5.5/2.54), subplot_kw={'projection': ccrs.PlateCarree()})
    ax_all = ax_all.flatten()
    
    data_mesh = BSS_horizon.sel(Method='EPEs_Indirect').values
    ax_all[0].pcolor(X_all, Y_all, data_mesh, transform=ccrs.PlateCarree(), 
                     norm=BoundaryNorm(Vals_used, len(Cols_used)), cmap=ListedColormap(Cols_used)) 

    ax_all[0].set_title("Forecasting horizon (in days) with indirect forecasts", pad=2, size=5, loc='left')

    # get the values so that the statistics are created and plotted on the colorbar
    AllData = data_mesh.flatten()[~np.isnan(data_mesh.flatten())]     
    Counts = pd.DataFrame({'Bins': [f'{i}~{j-1}' for i, j in zip(Vals_used[:-1], Vals_used[1:])], 
                           'Percentages': np.nan})
    Counts.iloc[-1, 0] = f'{Vals_used[-2]}~{Vals_used[-1]}'

    for i_bin in range(len(Counts)):
        if i_bin==len(Counts)-1:
            i_counts = sum((AllData >= Vals_used[i_bin]) & (AllData <= Vals_used[i_bin+1]))
        else:
            i_counts = sum((AllData >= Vals_used[i_bin]) & (AllData < Vals_used[i_bin+1]))
        Counts.iloc[i_bin, 1] = np.round(i_counts/len(data_mesh.flatten())*100, 2).astype(str)

    ax_sub = plot_discrete_bar_inset(col_lims=np.arange(len(Vals_used)), col_used=Cols_used, col_text='white',
                                     data_used=Counts.Percentages.values, loc=8, width='100%', height='15%',
                                     brdpad=-1.5, cat_names=Counts.Bins.values, bar_text_size=4.5, xlabel='',
                                     ax=ax_all[0])
    ax_sub.set_xticks([])
    ax_sub.set_xlabel('')
    [i.set_linewidth(0.1) for i in ax_sub.spines.values()]

    
    HorizonDif = BSS_horizon.fillna(0).diff('Method').values[0, ...]
    MaxDif = int(np.abs(HorizonDif).max())
    Colors_used = sns.color_palette('YlOrRd_r', n_colors=4) + ['white'] + sns.color_palette('GnBu', n_colors=4)
    Colors_limits = [-10, -6, -4, -2, 0, 1, 3, 5, 7, 10]
    if MaxDif>max(Colors_limits):
        Colors_limits[0] = -MaxDif
        Colors_limits[-1] = MaxDif

    ax_all[1].pcolor(X_all, Y_all, HorizonDif, transform=ccrs.PlateCarree(), 
                     norm=BoundaryNorm(Colors_limits, len(Colors_used)), cmap=ListedColormap(Colors_used)) 
    ax_all[1].set_title('Indirect-direct forecasting horizon difference (in days)', pad=2, size=5, loc='left')

    AllData = HorizonDif.flatten()  
    Counts = pd.DataFrame({'Bins': [f'{i}~{j-1}' for i, j in zip(Colors_limits[:-1], Colors_limits[1:])], 
                           'Percentages': np.nan})
    Counts.iloc[-1, 0] = f'{Colors_limits[-2]}~{Colors_limits[-1]}'
    Counts.iloc[4,0] = '0'

    for i_bin in range(len(Counts)):
        if i_bin==len(Counts)-1:
            i_counts = sum((AllData >= Colors_limits[i_bin]) & (AllData <= Colors_limits[i_bin+1]))
        else:
            i_counts = sum((AllData >= Colors_limits[i_bin]) & (AllData < Colors_limits[i_bin+1]))
        Counts.iloc[i_bin, 1] = np.round(i_counts/len(data_mesh.flatten())*100, 2).astype(str)

    ax_sub = plot_discrete_bar_inset(col_lims=np.arange(len(Colors_limits)), col_used=Colors_used, col_text='black',  
                                     data_used=Counts.Percentages.values, loc=8, width='100%', height='15%', 
                                     brdpad=-1.5, cat_names=Counts.Bins.values, bar_text_size=4.5, xlabel='',
                                     ax=ax_all[1])
    ax_sub.set_xticks([])
    ax_sub.set_xlabel('')
    [i.set_linewidth(0.1) for i in ax_sub.spines.values()]

    
    for ax in ax_all:
        ax.coastlines(resolution='110m', linewidth=.5, color='black')
        ax.set_extent([X.min(), X.max(), Y.min(), Y.max()], crs=ccrs.PlateCarree())

    plt.subplots_adjust(left=0.02, bottom=0.02, right=.98, top=1, hspace=.0, wspace=0.)
    
    return fig

In [43]:
GraphAb = GraphicalAbstract()
GraphAb.savefig(output_dir+'Pdf/GraphAb.pdf', dpi=600)
GraphAb.savefig(output_dir+'Png/GraphAb.png', dpi=600, facecolor=GraphAb.get_facecolor())
del(GraphAb, GraphicalAbstract)

In [44]:
del(Aggregated, AggregatedData, BSS_Combo, BSS_Ref, BS_EPEs, Cl_cols_used, Cl_vals_used, Cols_used,
    Vals_used_general, Weights, X, Y, X_all, Y_all, btlw, ffill_windowed)

### Figure 10

In [45]:
EV_EPEs = xr.open_dataarray(input_dir+f'ForecastsEPEs_Analysis/EV_leaddays.nc')
Weights = np.cos(np.radians(EV_EPEs.latitude))

In [46]:
Ref = EV_EPEs.sel(Method=['Clim_DayMonth', 'Clim_Seasonal']).max('Method').assign_coords({'Method': 'Reference'})
EV_EPEs = xr.concat([EV_EPEs.sel(Method=['Direct', 'Indirect']), Ref], dim='Method')
del(Ref)

In [47]:
AggregatedData = [aggregated_stats(EV_EPEs, i_val) for i_val in Areas.values()]
AggregatedData = xr.concat(AggregatedData, dim=pd.Index(list(Areas.keys()), name='Area'))

In [48]:
EV_Max = AggregatedData.max('Method')
EV_Max.name = 'Max'
EV_ArgMax = AggregatedData.argmax('Method')
EV_ArgMax.name = 'ArgMax'
EV_Max = xr.merge([EV_Max, EV_ArgMax])
del(EV_ArgMax)

In [49]:
EV_Max_DF = EV_Max.to_dataframe().reset_index()
EV_Max_DF = EV_Max_DF.sort_values(['Area', 'percentile', 'leaddays', 'cost_ratio'])

EV_Max_DF = EV_Max_DF.query('Max>0')
EV_Max_DF.index = range(len(EV_Max_DF))

# create extra rows when the argmax changes for the same lead time. These extra rows will be located in the half 
# distance of the previous and next, and have as argmax once the previous and once the next argmax, so consecutive
# lines without gaps, and with the color of the relevant method, can be constructed
EV_Max_DF['ArgMax2'] = EV_Max_DF['ArgMax'].shift()
EV_Max_DF.loc[0, 'ArgMax2'] = EV_Max_DF.loc[0, 'ArgMax']

mask = (EV_Max_DF['ArgMax2']!=EV_Max_DF['ArgMax']) & (EV_Max_DF['leaddays']==EV_Max_DF['leaddays'].shift())
df_change = EV_Max_DF[mask].copy(deep=True)
df_change[['cost_ratio', 'Max', 'ArgMax', 'ArgMax2']] = np.nan
df_change = df_change.set_index(df_change.index-0.5)
for i_ind in df_change.index:
    values_used = EV_Max_DF.loc[[np.floor(i_ind), np.ceil(i_ind)], ['cost_ratio', 'Max']].mean()
    df_change.loc[i_ind, ['cost_ratio', 'Max']] = values_used
    
EV_Max_DF = pd.concat([EV_Max_DF, df_change, df_change]).sort_index().drop('ArgMax2', 1)
EV_Max_DF.ArgMax = EV_Max_DF.ArgMax.fillna(method='ffill', limit=1)
EV_Max_DF.ArgMax = EV_Max_DF.ArgMax.fillna(method='bfill', limit=1)
del(i_ind, values_used, mask, df_change)

In [50]:
def Fig10_type(perc_used=95, lead_times_used=[1, 5, 7, 14]):
    
    DF_used = EV_Max_DF.query('percentile==@perc_used and leaddays==@lead_times_used')
    Max = np.ceil(DF_used.Max.max()*10)/10
    fig, ax_all = plt.subplots(2, 3, figsize=(18/2.54, 9/2.54))
    ax_all = ax_all.flatten()
    
    for i, i_area in enumerate(Areas):
        for j, i_lead in enumerate(lead_times_used):
            DF_new = DF_used.query('Area==@i_area and leaddays==@i_lead')
            DF_new = DF_new.pivot_table(values='Max', index='cost_ratio', columns='ArgMax')
            DF_new = DF_new.reindex(DF_new.columns.union(np.arange(len(EV_EPEs.Method)).astype(float)), axis=1)
            DF_new.columns = pd.Index(EV_EPEs.Method, name='Method')
            if j>0:
                DF_new.plot(linewidth=2, color=sns.color_palette(n_colors=4), ax=ax_all[i], legend=False)
            else:
                DF_new.plot(linewidth=2, color=sns.color_palette(n_colors=4), ax=ax_all[i], legend=True)
                if i==0: [line.set_linewidth(2) for line in ax_all[i].legend().get_lines()]
        
        if i!=0: ax_all[i].legend().remove()
        
        ax_all[i].set_xlabel('Cost-Loss ratio')
        ax_all[i].set_ylim(0, Max)
        ax_all[i].set_xlim(0, 1)
        ax_all[i].set_ylabel('Vmax')
        ax_all[i].set_title(f"({['a', 'b', 'c', 'd', 'e', 'f'][i]}) {i_area}", pad=4, loc='left', size=8)
        ax_all[i].xaxis.set_tick_params(width=.25, length=2) # modify x-axis ticks
        ax_all[i].yaxis.set_tick_params(width=.25, length=2) # modify y-axis ticks
        
        if i>0:
            ax_sub = inset_axes(ax_all[i], loc=1, width='60%', height='40%',
                                axes_class=cartopy.mpl.geoaxes.GeoAxes, 
                                axes_kwargs=dict(map_projection=ccrs.PlateCarree()))
            ax_sub.coastlines(resolution='110m', linewidth=.5, color='black')
            ax_sub.set_extent([X_min, X_max, Y_min, Y_max], crs=ccrs.PlateCarree())
            x_min, x_max, y_min, y_max = Areas[i_area]
            rect = plt.Rectangle((x_min, y_min),x_max-x_min, y_max-y_min, fill=False, color='red', linewidth=.75) 
            ax_sub.add_patch(rect) # add rectangle with the actual area used for the Precipitation analysis

            
    plt.subplots_adjust(left=0.07, bottom=0.12, right=.99, top=.93, wspace=0.3, hspace=0.5)
    return fig

In [51]:
for i_perc in [90, 95, 99]:
    Fig10 = Fig10_type(perc_used=i_perc, lead_times_used=[7, 11, 17])
    Fig10.savefig(output_dir+f'Pdf/Fig10_{i_perc}.pdf')
    Fig10.savefig(output_dir+f'Png/Fig10_{i_perc}.png', dpi=600, facecolor=Fig10.get_facecolor())
    if i_perc!=95: plt.close()
    
del(i_perc, Fig10, Fig10_type)

In [52]:
def FigS10_type_supl(perc_used=95, lead_times_used=11):
    
    DF_used = AggregatedData.sel(percentile=perc_used, leaddays=lead_times_used)
    
    Max = np.ceil(DF_used.values.max()*10)/10
    fig, ax_all = plt.subplots(2, 3, figsize=(18/2.54, 9/2.54))
    ax_all = ax_all.flatten()
    
    for i, i_area in enumerate(Areas):
        DF_new = DF_used.sel(Area=i_area).to_dataframe('Max').reset_index().query('Max>0')
        sns.lineplot(data=DF_new, x='cost_ratio', y='Max', hue='Method', linewidth=2, ax=ax_all[i])

        if i!=0: ax_all[i].legend().remove()
        if i==0: [line.set_linewidth(2) for line in ax_all[i].legend().get_lines()]
    
        ax_all[i].set_xlabel('Cost-Loss ratio')
        ax_all[i].set_ylim(0, Max)
        ax_all[i].set_xlim(0, 1)
        ax_all[i].set_ylabel('Vmax')
        ax_all[i].set_title(f"({['a', 'b', 'c', 'd', 'e', 'f'][i]}) {i_area}", pad=4, loc='left', size=8)
        ax_all[i].xaxis.set_tick_params(width=.25, length=2) # modify x-axis ticks
        ax_all[i].yaxis.set_tick_params(width=.25, length=2) # modify y-axis ticks
        
        if i>0:
            ax_sub = inset_axes(ax_all[i], loc=1, width='60%', height='40%', #bbox_to_anchor=(0, 1, 1, 1),
                                axes_class=cartopy.mpl.geoaxes.GeoAxes, 
                                axes_kwargs=dict(map_projection=ccrs.PlateCarree()))
            ax_sub.coastlines(resolution='110m', linewidth=.5, color='black')
            ax_sub.set_extent([X_min, X_max, Y_min, Y_max], crs=ccrs.PlateCarree())
            x_min, x_max, y_min, y_max = Areas[i_area]
            rect = plt.Rectangle((x_min, y_min),x_max-x_min, y_max-y_min, fill=False, color='red',# linestyle='--',
                                 linewidth=.75) 
            ax_sub.add_patch(rect) # add rectangle with the actual area used for the Precipitation analysis

            
    plt.subplots_adjust(left=0.07, bottom=0.12, right=.99, top=.93, wspace=0.3, hspace=0.5)
    return fig

In [53]:
for i_perc in [95]:
    for i_lead in [7]:
        FigS10 = FigS10_type_supl(perc_used=i_perc, lead_times_used=i_lead)
        FigS10.savefig(output_dir+f'Pdf/FigS10_{i_perc}_{i_lead}.pdf')
        FigS10.savefig(output_dir+f'Png/FigS10_{i_perc}_{i_lead}.png', dpi=600, facecolor=FigS10.get_facecolor())
        if i_perc!=95 or i_lead!=7: plt.close()
    
del(i_perc, i_lead, FigS10, FigS10_type_supl)

In [54]:
del(AggregatedData, EV_EPEs, EV_Max, EV_Max_DF, Weights, aggregated_stats)